### Loading Json data 
First, we load the staircase data from the .json file, in a dictionary. <br>
Both modules (json, pprint) are included in the standard python installation.  

In [1]:
import json
from pprint import pprint

# Read the JSON file and parse it into a list of dictionaries
with open("../IFC Conversion/stairs.json", "r") as json_file:
    stairs_dict = json.load(json_file)

pprint(stairs_dict)

[{'NumberOfRailings': 2,
  'NumberOfRiser': 16,
  'RiserHeight': 175,
  'TreadLength': 240,
  'id': '2N1$wSskz2bAjjtl_KLXOJ',
  'name': 'Precast Stair:Stair:381758'},
 {'NumberOfRailings': 2,
  'NumberOfRiser': 18,
  'RiserHeight': 178,
  'TreadLength': 240,
  'id': '1qnuHei2H9IuA8Q46m1Dkm',
  'name': 'Precast Stair:Stair:383689'},
 {'NumberOfRailings': 2,
  'NumberOfRiser': 3,
  'RiserHeight': 107,
  'TreadLength': 240,
  'id': '06M_CRzcPB5P5z43s0BDRD',
  'name': 'Precast Stair:Stair:397996'}]


### Connect to the KIE server 
To check our data against the DMN rules, we connect to the KIE server running in the background. This service acts as a reasoning engine that can also be accessed manually by entering the IP address (found in the console) in a browser. This can be a good first check of your DMN model.
We use the 'requests' module, which we *pip install requests* in the terminal. <br>
As a starting point, let's see if we can ping the endpoint by using the ip address provided by the KIE server...<br>
Let's send an empty json file as the post content, this will return an *internal server* error (status code:500), which is fine because it also signals that we have managed to connect.

In [2]:

import requests
import os

endpoint = 'http://127.0.0.1:21345/jitdmn' 
response = requests.post(endpoint, {}, headers={"Content-Type": "application/json"})

# Print the status code
print(f"Status Code: {response.status_code}")

Status Code: 500


### Evaluating project data against decision models
OK, with the connection established, we define a function to manage communication, it takes three arguments: 
- dmn model,
- server edpoint,
- required project data. <br>



In [5]:
# Function to send file and data
def send_file_and_data(dmn, endpoint, data):
    try:
        # Read the XML file
        with open(dmn, 'r', encoding='utf-8') as file:
            xml_data = file.read()

        # Create the request body
        body = {
            "model": xml_data,
            "context": data
        }

        # Send the POST request
        response = requests.post(endpoint, json=body, headers={
            "Content-Type": "application/json"
        })
        
        # Print the response
        print('Response:', response.json())
    except Exception as error:
        print('Error sending file and data:', error)

dmn = os.path.join(os.getcwd(), 'StairIsAccessible.dmn') 
for s in stairs_dict:
    send_file_and_data(dmn, endpoint, s)


Response: {'StairFormula': 'function StairFormula( rh, tl )', 'StairFormulaOk': True, 'TreadLength': 240, 'RailingsOk': True, 'StairIsAccessible': True, 'NumberOfRiser': 16, 'name': 'Precast Stair:Stair:381758', 'RiserHeight': 175, 'NumberOfRailings': 2, 'id': '2N1$wSskz2bAjjtl_KLXOJ'}
Response: {'StairFormula': 'function StairFormula( rh, tl )', 'StairFormulaOk': True, 'TreadLength': 240, 'RailingsOk': True, 'StairIsAccessible': True, 'NumberOfRiser': 18, 'name': 'Precast Stair:Stair:383689', 'RiserHeight': 178, 'NumberOfRailings': 2, 'id': '1qnuHei2H9IuA8Q46m1Dkm'}
Response: {'StairFormula': 'function StairFormula( rh, tl )', 'StairFormulaOk': False, 'TreadLength': 240, 'RailingsOk': True, 'StairIsAccessible': False, 'NumberOfRiser': 3, 'name': 'Precast Stair:Stair:397996', 'RiserHeight': 107, 'NumberOfRailings': 2, 'id': '06M_CRzcPB5P5z43s0BDRD'}
